1. Importar las bibliotecas necesarias:

In [3]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
import re

2. Cargar y limpiar el archivo CSV:

In [23]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv("tweets_2022_abril_junio.csv", nrows=500000) # Limitar la cantidad de filas para manejar la memoria

# Limpiar el texto de los tweets
df["text"] = df["text"].str.replace("\n", " ") # Reemplazar saltos de línea por espacios
df["text"] = df["text"].str.lower() # Convertir el texto a minúsculas

# Eliminar tweets que empiezan por "rt"
df = df[~df["text"].str.startswith("rt")]

# Eliminar emojis
df["text"] = df["text"].apply(lambda x: re.sub(r"\W+", " ", x.encode('ascii', 'ignore').decode('utf-8')))

# Eliminar hashtags
df["text"] = df["text"].str.replace(r"#\w+", "")

# Eliminar filas con texto vacío o solo espacios
df = df.dropna(subset=["text"])
df = df[~df["text"].str.isspace()]
df = df[df["text"].str.len() > 3]  # Usar str.len() para filtrar filas con texto vacío

# Borramos textos duplicados
df.drop_duplicates(subset="text", keep="first", inplace=True)

# Borramos filas que contengan texto "http"
df = df[~df["text"].str.contains("http")]

# Imprime la cantidad de filas del DataFrame
print("Cantidad de filas:", len(df)) # Raw: 4594980, quitando RT: 1267332

# Guardamos el DataFrame en un archivo CSV
df.to_csv("tweets_2022_abril_junio_clean.csv", index=False)

C:\Users\herna\AppData\Local\Temp\ipykernel_34756\1968410213.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r"#\w+", "")


Cantidad de filas: 137177


In [24]:
def get_k_shingles(text, k=4):
    return [text[i:i+k] for i in range(len(text) - k + 1)]

# Crear una lista para almacenar los MinHash de cada tweet
minhashes = []

# Crear MinHash para cada tweet
for text in df["text"]:
    m = MinHash(num_perm=128)
    for shingle in get_k_shingles(text):
        m.update(shingle.encode("utf8"))
    minhashes.append(m)

# Crear LSH con un umbral de similitud s
s = 0.7
lsh = MinHashLSH(threshold=s, num_perm=128)

# Agregar MinHashes al LSH
for i, m in enumerate(minhashes):
    lsh.insert(i, m)

similar_tweets = [] # Lista para almacenar los pares de tweets similares

# Iterar sobre cada tweet
for i, m in enumerate(minhashes):
    # Consultar el LSH para encontrar tweets similares
    similar_indices = lsh.query(m)
    
    # Iterar sobre los índices de los tweets similares
    for j in similar_indices:
        # Si los usuarios son diferentes y el índice j es mayor que i
        if df.iloc[i]["screen_name"] != df.iloc[j]["screen_name"] and j > i:
            similar_tweets.append((i, j))

# Crear un DataFrame con los pares de tweets similares
similar_tweets_df = pd.DataFrame(similar_tweets, columns=["Index1", "Index2"])

# Añadir información de usuario y texto al DataFrame
similar_tweets_df["User1"] = similar_tweets_df["Index1"].apply(lambda x: df.iloc[x]["screen_name"])
similar_tweets_df["User2"] = similar_tweets_df["Index2"].apply(lambda x: df.iloc[x]["screen_name"])
similar_tweets_df["Tweet1"] = similar_tweets_df["Index1"].apply(lambda x: df.iloc[x]["text"])
similar_tweets_df["Tweet2"] = similar_tweets_df["Index2"].apply(lambda x: df.iloc[x]["text"])

# Guardar el DataFrame en un archivo CSV
similar_tweets_df.to_csv("similar_tweetsP.csv", index=False)

In [ ]:
similar_tweets_df = pd.read_csv("similar_tweets.csv")

user_pairs_count = similar_tweets_df.groupby(["User1", "User2"]).size().reset_index(name="Count")

user_pairs_sorted = user_pairs_count.sort_values(by="Count", ascending=False)

top_user_pairs = user_pairs_sorted.head(10) # Mostrar los 10 primeros pares
print(top_user_pairs)